#  Модель ML, помогающая находить элементы, которые могут быть успешно исполнены спортсменом на соревновании.

**Заказчик**

Компания `GoProtect`

**Описание проекта**

Сервис  `Мой Чемпион` помогает спортивным школам фигурного катания, тренерам мониторить результаты своих подопечных и планировать дальнейшее развитие спортсменов.

**Цель**

Создать модель, помогающую находить элементы, которые могут быть успешно исполнены спортсменом на соревновании. 
Сервис будет прогнозировать прогресс и возможное выполнение тех или иных элементов программы по истории предыдущих выступлений и выполнения элементов на соревнованиях.


В работе будет сделан упер на прогнозирование ошиок в прыжках и сделан отбор только спорсменов, которые принимали участие в полных соревнованиях (короткая и произвольные программмы).

&copy; by Копцев Андрей [<img src="https://upload.wikimedia.org/wikipedia/commons/8/82/Telegram_logo.svg" alt="Telegram" width="17"/>](https://t.me/Andrei_VK_1980)

## Импорты

In [1]:
# системные модули
import os
import sys
import warnings
import re
import pdfplumber
import numpy as np
import pandas as pd
import unicodedata
from colorama import Back, Fore, Style

from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.metrics import f1_score, classification_report
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.multioutput import MultiOutputClassifier

from lightgbm import LGBMClassifier

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
pd.set_option("max_colwidth", 100)

warnings.filterwarnings("ignore")

**Переменные**

In [2]:
from data_loader import GetDataframe
RANDOM = 12345
# рабочая директория проекта
WORK_DIR = os.path.abspath(os.curdir)
# директория c датасетами
DATA_DIR = os.path.join(WORK_DIR, "datasets")
ADD_INFO_DIR = os.path.join(WORK_DIR, "add_info")
ELEMS_FILE = "SSPScomm_2475.pdf"
# директория с модулями
MODULES_DIR = os.path.join(WORK_DIR, "modules")
# добавляем папку с модулями в систему
sys.path.append(MODULES_DIR)
# файлы с данными для датасетов
TOTAL_SCORES_FILE = "total_scores.csv"
TOURNAMENT_SCORES_FILE = "tournament_scores.csv"
TOURNAMENTS_FILE = "tournaments.csv"
UNITS_FILE = "units.csv"

## Загрузка данных

**Инициализация объекта класса.**

In [3]:
df_loader = GetDataframe(DATA_DIR)

Загружаем датафрейм с оценками за выступления по категориям и общие за турнир

In [4]:
total_scores = df_loader.get_dataframe_from_file(file=TOTAL_SCORES_FILE)

Создаем датасет из файла total_scores.csv.
Датасет создан!
Время выполнения: 0.08 секунд(ы)


Информация по датафрейму

In [5]:
df_loader.get_dataframe_info(dataset=total_scores)

Общая информация по датасету
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21301 entries, 0 to 21300
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   21301 non-null  int64  
 1   unit_id              21301 non-null  int64  
 2   tournament_id        21301 non-null  int64  
 3   base_score           21301 non-null  float64
 4   components_score     21301 non-null  float64
 5   total_score          21301 non-null  float64
 6   elements_score       21301 non-null  float64
 7   decreasings_score    21301 non-null  float64
 8   starting_place       21301 non-null  int64  
 9   place                21301 non-null  int64  
 10  segment_name         21284 non-null  object 
 11  info                 20720 non-null  object 
 12  overall_place        21301 non-null  int64  
 13  overall_total_score  21284 non-null  float64
 14  overall_place_str    10814 non-null  object 
dtypes: floa

,percent_missing
segment_name,0.079808
info,2.727571
overall_total_score,0.079808
overall_place_str,49.232430


,id,unit_id,tournament_id,base_score,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str
13700,8379,1363,40,39.90,54.15,90.69,38.54,0.0,7,3,Произвольная программа,q Прыжок приземлён в четверть < Недокрученный прыжок x Надбавка за прыжки во второй половине про...,3,138.41,NaN
15829,12492,2050,77,20.13,25.49,44.24,19.75,0.0,31,10,Короткая программа,q Прыжок приземлён в четверть < Недокрученный прыжок x Надбавка за прыжки во второй половине про...,12,111.93,NaN
11912,5738,605,19,67.66,83.02,140.41,60.39,0.0,9,13,Произвольная программа,< Недокрученный прыжок << Пониженный прыжок ! Неясное ребро на толчке F/Lz x Надбавка за прыжки ...,11,233.68,NaN
15776,12444,117,77,17.90,25.42,42.60,18.18,0.0,32,20,Короткая программа,< Недокрученный прыжок << Пониженный прыжок x Надбавка за прыжки во второй половине программы (1...,14,117.52,NaN
717,442658,1,4844,28.96,28.00,60.70,32.70,0.0,4,3,Короткая программа,! Неясное ребро на толчке F/Lz x Надбавка за прыжки во второй половине программы (10%),3,183.24,3


Датафрейм состоит из следующих полей:

➡️ `id`: идентификатор выступления.

➡️ `unit_id`: идентификатор юнита.

➡️ `tournament_id`: идентификатор турнира.

➡️ `components_score`: артистизм (мастерство, композиция, хореография).

➡️ `base_score`: базовая оценка за элементы в выступлении (идеал).

➡️ `elements_score`: реальная оценка всех выполненных элементов, base_score+goe.

➡️ `decreasings_score`: снижения оценок за ошибки.

➡️ `total_score`: `components_score+elements_score+decreasings_score` за выступление.

➡️ `starting_place`: номер по порядку при начаде выступленияю.

➡️ `place`: занятое место в категории `category_name+segment_name`.

➡️ `segment_name`: название сегмента.

➡️ `info`: комментарии и пояснения к оценке.

➡️ `overall_place`: итоговое место в турнире.

➡️ `overall_total_score`: итоговая оценка за весь турнир.

➡️ `overall_place_str`: комментарии, пояснения.

Есть пропуски, но от них избавимся в процессе объединения всех датасетов.

Загружаем датафрейм с оценками поэлементно.

In [6]:
tournament_scores = df_loader.get_dataframe_from_file(file=TOURNAMENT_SCORES_FILE)

Создаем датасет из файла tournament_scores.csv.
Датасет создан!
Время выполнения: 0.11 секунд(ы)


Информация по датафрейму

In [7]:
df_loader.get_dataframe_info(dataset=tournament_scores)

Общая информация по датасету
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172158 entries, 0 to 172157
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              172158 non-null  int64  
 1   total_score_id  172158 non-null  int64  
 2   title           172158 non-null  object 
 3   decrease        41185 non-null   object 
 4   base_score      172158 non-null  float64
 5   goe             172158 non-null  float64
 6   avg_score       172158 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 9.2+ MB

Столбцы с пропущенными значениями:


,percent_missing
decrease,76.077208


,id,total_score_id,title,decrease,base_score,goe,avg_score
147336,272857,454558,1F,NaN,0.5,0.03,0.53
144677,270290,454241,CCoSp4,NaN,3.5,0.70,4.20
166430,350665,463223,3Loq,q,0.0,-0.39,5.00
62686,72009,12850,1A,F,1.1,-0.55,0.55
88945,167214,443384,LSp3,NaN,2.4,0.32,2.72


Датафрейм состоит из следующих полей:

➡️ `id`: идентификатор оценки за конкретный элемент/комбинацию.

➡️ `total_score_id`: идентикатор выступления.

➡️ `title`: запись элемента или комбинации элементов с отметками об ошибках.

➡️ `decrease`: за что снижена оценка.

➡️ `base_score`: базовая оценка (идеал, цена данного элемента/комбинации, сложность).

➡️ `goe`: `Grade of Execute`, качество исполнения, судейские надбавки/убавки.

➡️ `avg_score`: оценка за элемент/комбинацию (усредненная по судьям).

Есть пропуски, но от них избавимся в процессе объединения всех датасетов.

Загружаем датафрейм с турнирами. 

Турнир состоит из нескольких категорий, оценки по категориям расписаны в `total_scores`.

In [8]:
tournaments = df_loader.get_dataframe_from_file(file=TOURNAMENTS_FILE)

Создаем датасет из файла tournaments.csv.
Датасет создан!
Время выполнения: 0.01 секунд(ы)


Информация по датафрейму

In [9]:
df_loader.get_dataframe_info(dataset=tournaments)

Общая информация по датасету
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          142 non-null    int64  
 1   date_start  142 non-null    object 
 2   date_end    142 non-null    object 
 3   origin_id   142 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 4.6+ KB

В датасете нет пропущенных значений.


,id,date_start,date_end,origin_id
7,8,2090-11-02,2090-11-04,2.0
47,50,2091-04-21,2091-04-23,2.0
73,176,2091-03-21,2091-03-23,2.0
141,6676,2092-02-28,2092-03-01,1.0
38,41,2091-03-14,2091-03-17,2.0


Датафрейм состоит из следующих полей:

➡️ `id`: идентификатор турнира.

➡️ `date_start`: дата начала.

➡️ `date_end`: дата завершения.

➡️ `origin_id`: место проведения.

Есть пропуски, но от них избавимся в процессе объединения всех датасетов.

Загружаем датафрейм со спортсменами. 

Юнит – спортивная единица. Мы работаем с одиночным катанием, в одном юните только один
спортсмен. Если бы были пары/команды, то было б несколько спортсменов в юните.

In [10]:
units = df_loader.get_dataframe_from_file(file=UNITS_FILE)

Создаем датасет из файла units.csv.
Датасет создан!
Время выполнения: 0.01 секунд(ы)


Информация по датафрейму

In [11]:
df_loader.get_dataframe_info(dataset=units)

Общая информация по датасету
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4596 entries, 0 to 4595
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         4596 non-null   int64  
 1   color      4595 non-null   object 
 2   school_id  4007 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 107.8+ KB

Столбцы с пропущенными значениями:


,percent_missing
color,0.021758
school_id,12.815492


,id,color,school_id
2704,4268,green,285.0
2147,3270,green,254.0
3272,34854,green,256.0
2983,4239,green,5.0
2113,9613,green,184.0


Датафрейм состоит из следующих полей:

➡️ `id`: идентификатор юнита.

➡️ `color`: категория.

➡️ `school_id`: идентификатор школы.

Есть пропуски, но от них избавимся в процессе объединения всех датасетов.

Датафреймы созданы.

## Объединение датафреймов

Для объединения датафреймов будет использована функция `merge_df`. Логика обработки следующая.

Чтобы избежать появления дублированных столбцов при объединении нужно привести ключевые столцы к одинаковым названиям:

🛠 В `total_scores` переименовываем `id` в `total_score_id`.

🛠 В `tournament_scores` переименовываем `id` в `tournament_scores_id`.

🛠 В `tournaments` переименовываем `id` в `tournament_id`.

🛠 В `units` переименовываем `id` в `unit_id`.

Объединяем последовательно в результирующий датафрейм `data` (все объдинения `inner`): 

🛠 Сначала объединяем `total_scores` и `tournament_scores` по `total_score_id`.

🛠 Затем объединяем с `units` по `unit_id`.

🛠 И наконец объединяем с `tournaments` по `tournament_id`.

В обединненом датафреме переименовываем столбцы `base_score_x` в `base_score`, `base_score_y` в `base_score_elements`.

Помимо этого в столбце `segment_name` из `total_scores` есть вот такой расклад:

In [12]:
total_scores["segment_name"].unique()

array(['Короткая программа', 'Произвольная программа',
       'Произвольная программа.', 'Элементы', 'Интерпретация',
       'Интерпретация.', nan, 'Кoроткая программа',
       'Произвольнaя программа', 'Пpoизвoльнaя пpoгpaммa',
       'Кopoткaя пpoгpaммa', 'Элeмeнты', 'Коpоткая программа',
       'Пpoизвольная программа', 'Прыжки', 'Kopoткaя пpoгpaммa',
       'Пpoизвольнaя программа', 'Прoизвольная прoграмма'], dtype=object)

Нужно привести к уникальным названиям.

И в финале оставим только `Kopoткaя пpoгpaммa` и  `Произвольная программа`, т.к. рассатривать будем только полноценные соревнования.

In [13]:
def merge_df(total_scores_df, tournament_scores_df, tournaments_df, units_df):
    """ Функция для объединения и предобработки датафремов

    Параметры:
    - total_scores_df (pd.DataFrame): датафрейм total_scores.
    - tournament_scores_df (pd.DataFrame): датафрейм tournament_scores.
    - tournaments_df (pd.DataFrame): датафрейм tournaments.
    - units_df (pd.DataFrame): датафрейм units_df.

    Возвращает:
    - data (pd.DataFrame): объединенный датафрейм.

    """
    # Переименование столбцов для объединения
    total_scores_df = total_scores_df.rename(columns={"id": "total_score_id"})
    tournament_scores_df = tournament_scores_df.rename(
        columns={"id": "tournament_scores_id"})
    tournaments_df = tournaments_df.rename(columns={"id": "tournament_id"})
    units_df = units_df.rename(columns={"id": "unit_id"})
    # Объединение датафреймов
    data = total_scores_df.merge(
        tournament_scores_df, on="total_score_id", how="inner")
    data = data.merge(units_df, on="unit_id", how="inner")
    data = data.merge(tournaments_df, on="tournament_id", how="inner")
    # Переименование дубликатных столбцов
    data = data.rename(
        columns={"base_score_x": "base_score", "base_score_y": "base_score_elements"})
    # Выбор необходимых столбцов
    cols_structure = ["total_score_id", "unit_id", "color", "tournament_id", 
                      "date_start", "date_end", "origin_id",
                      "base_score", "components_score", "total_score", 
                      "elements_score", "decreasings_score", "starting_place",
                      "place", "segment_name", "info", "overall_place", "overall_total_score", 
                      "title", "decrease", "base_score_elements", "goe", "avg_score"]
    data = data[cols_structure]
    # Сортировка по дате начала
    data = data.sort_values(by="date_start", ascending=True)
    # Стандартизация названий сегментов
    data.loc[data["segment_name"].isin(["Короткая программа",
                                        "Кoроткая программа", "Кopoткaя пpoгpaммa", "Kopoткaя пpoгpaммa",
                                        "Коpоткая программа"]), "segment_name"] = "Короткая программа"
    data.loc[data["segment_name"].isin(["Произвольная программа",
                                        "Произвольная программа.", "Произвольнaя программа",
                                        "Пpoизвoльнaя пpoгpaммa", "Прoизвольная прoграмма", "Пpoизвольная программа",
                                        "Пpoизвольнaя программа"]), "segment_name"] = "Произвольная программа"
    data.loc[data["segment_name"].isin(
        ["Интерпретация.", "Интерпретация'"]), "segment_name"] = "Интерпретация"
    data.loc[data["segment_name"].isin(
        ["Элементы", "Элeмeнты"]), "segment_name"] = "Элементы"
    # Оставляем только строки с "Короткой программой" или "Произвольной программой"
    data = data[(data["segment_name"] == "Короткая программа") |
                (data["segment_name"] == "Произвольная программа")]
    # Группируем данные по ['unit_id', 'tornament_id'] и проверьте, есть ли как "short", так и "free" для каждой группы
    grouped = data.groupby(["unit_id", "tournament_id"])
    mask = grouped["segment_name"].transform(lambda x: set(
        x).issuperset({"Короткая программа", "Произвольная программа"}))
    # Оставляем только строки, соответствующие маске
    data = data[mask]
    # Объединенный датафрейм
    return data

Получаем объединенный датасет.

In [14]:
data = merge_df(total_scores, tournament_scores, tournaments, units)

Информация по датафрейму

In [15]:
df_loader.get_dataframe_info(dataset=data)

Общая информация по датасету
<class 'pandas.core.frame.DataFrame'>
Index: 133637 entries, 23451 to 58285
Data columns (total 23 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   total_score_id       133637 non-null  int64  
 1   unit_id              133637 non-null  int64  
 2   color                133637 non-null  object 
 3   tournament_id        133637 non-null  int64  
 4   date_start           133637 non-null  object 
 5   date_end             133637 non-null  object 
 6   origin_id            133637 non-null  float64
 7   base_score           133637 non-null  float64
 8   components_score     133637 non-null  float64
 9   total_score          133637 non-null  float64
 10  elements_score       133637 non-null  float64
 11  decreasings_score    133637 non-null  float64
 12  starting_place       133637 non-null  int64  
 13  place                133637 non-null  int64  
 14  segment_name         133637 non-null  obj

,percent_missing
info,0.897207
decrease,78.121329


,total_score_id,unit_id,color,tournament_id,date_start,date_end,origin_id,base_score,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,title,decrease,base_score_elements,goe,avg_score
45480,463160,602,green,7115,2092-04-23,2092-04-26,2.0,53.28,46.85,89.39,46.54,-4.0,5,11,Произвольная программа,q Прыжок приземлён в четверть REP Повторение прыжка не в каскаде x Надбавка за прыжки во второй ...,11,148.51,CCoSp4,NaN,3.5,0.63,4.13
109142,8336,803,green,40,2091-03-12,2091-03-15,2.0,48.14,48.48,94.91,46.43,0.0,35,7,Произвольная программа,< Недокрученный прыжок ! Неясное ребро на толчке F/Lz REP Повторение прыжка не в каскаде x Надба...,5,152.57,LSp4,NaN,2.7,0.54,3.24
94252,458341,9903,green,7111,2092-04-11,2092-04-13,2.0,49.32,52.07,101.58,51.51,-2.0,24,7,Произвольная программа,! Неясное ребро на толчке F/Lz REP Повторение прыжка не в каскаде x Надбавка за прыжки во второй...,7,157.27,3F!,F,0.0,-2.65,3.18
159109,7666,454,green,35,2091-02-26,2091-02-28,2.0,32.01,31.48,71.65,40.17,0.0,23,1,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),1,206.78,CCoSp4,NaN,3.5,1.05,4.55
35660,446358,9865,lime,6160,2092-01-29,2092-01-30,2.0,19.33,19.52,37.40,17.88,0.0,10,10,Короткая программа,"Представление@@@1.50@@@4,00@@@4,00@@@4,25@@@4,50@@@4,25@@@4.17",10,112.01,FCSp4,NaN,3.2,0.00,3.20


Есть пропуски в столбцах `info` и `decrease`. В `info` видимо нечего было писать, а в `decrease` пропуски образовываются только там, где снижение отсутсвует. В дальнейшем при аггрегации эти столбцы просто исчезнут.

## Финальный датафрейм

Получение финального датасета будет происходить в несколько этапов:

🪛 Из столбца `title` создадим столбец `clean_elems` - столбец с элементами без ошибок при помощи вспомогательной функции `clean_elements`.

🪛 Создадим дополнительный признак `max_base_score` - максимальный базовый скор для элементов. Базовы скор. можно взять из файла `SSPScomm_2475.pdf` при помощи вспомогательной функции `parse_pdf` и функции `max_base_score_for_clean_elements` (Источник: https://fsrussia.ru/files/docs/SSPScomm_2475.pdf).

🪛 Создадим первый аггрегированный датасет по спортсмену и турниру.

🪛 Создадим финальный датасет с агрегированной статистикой по спортсмену.

### Вспомогательная функция `convert_to_latin`

В датафрейме в написании типа прыжков, например `3A` может быть как латинский, так и кириллический символ. Для этого можно реализовать преобразование сходных по написанию символов из кириллицы в латиницу.

In [16]:
def convert_to_latin(string):
    """
    Функция для конвертации текста из кириллицы в латиницу.

    Параметры:
    - string (str): текст, который необходимо конвертировать.

    Возвращает:
    - latinic_text (str): конвертированный текст в латинице.
    """
    # Словарь символов кириллицы и их латинское соответствие
    char_mappings = {
        "А": "A",
        "а": "a",
        "О": "O",
        "о": "o",
        "Е": "E",
        "е": "e",
        "С": "C",
        "с": "c",
        "Т": "T",
        "М": "M",
        "Р": "P",
        "р": "p",
        "Х": "X",
        "х": "x",
    }
    # Используем регулярные выражения для замены кириллических символов на латинские
    latinic_text = re.sub("|".join(char_mappings.keys()),
                          lambda m: char_mappings[m.group()], string)
    # возвращает конвертированный текст в латинице.
    return latinic_text

### Вспомогательные функция `parse_pdf`, `clean_elements` и `max_base_score_for_clean_elements`

Функция `parse_pdf` распарсивает файл со скорами и на выходе возвращает словарь, где ключ это элемент, а значение базовый скор.

In [17]:
def parse_pdf(pdf_dir=None, file=None, convert_to_latin=None):
    """
    Функция для парсинга PDF-файла и извлечения базовых баллов элементов фигурного катания.

    Параметры:
    - pdf_dir (str): директория, в которой находится PDF-файл (по умолчанию None).
    - file (str): имя PDF-файла (по умолчанию None).
    - convert_to_latin (func) - функция конвертации кириллицы в латиницу (по умолчанию None).

    Возвращает:
    - elem_base_scores (dict): cловарь, где ключи - названия элементов фигурного катания, а значения - их базовые баллы.
    """
    # Проверка, что значения переменных не None
    if pdf_dir is None or file is None or convert_to_latin is None:
        raise ValueError(f"Переменные pdf_dir или file не должны быть пустыми")
    # Открываем PDF-файл и извлекаем таблицы
    with pdfplumber.open(os.path.join(pdf_dir, file)) as pdf:
        res_lst = []
        res_lst_final = []
        for i in range(1, 7):
            page = pdf.pages[i]
            table = page.extract_tables()[1:]
            table_list = [
                inner_list for sublist in table for inner_list in sublist]
            res_lst.extend(table_list)
    # Обрабатываем извлеченные данные
    elem_base_scores = dict()
    for item in res_lst:
        item_key = item[0]
        item_values = item[1:]
        item_values = [val.replace(",", ".") for val in item_values]
        float_values = [float(val) for val in item_values if len(val) > 0]
        max_value = max(float_values)
        elem_base_scores[item_key] = max_value
    # Добавляем отсутсвующие базовые баллы элементов
    elem_base_scores["1W"] = 0.2
    elem_base_scores["ChSq"] = 0.0
    elem_base_scores["ChSq1*"] = 0.0
    elem_base_scores["ChSpl1"] = 1.5
    elem_base_scores["ChSpl"] = 0.0
    elem_base_scores["FSSp"] = 0.0
    elem_base_scores["CCSp"] = 0.0
    elem_base_scores["FCSp"] = 0.0
    elem_base_scores["CSSp"] = 0.0
    elem_base_scores["CoSp"] = 0.0
    elem_base_scores["Sp"] = 0.0
    elem_base_scores["CSp"] = 0.0
    elem_base_scores["SSp"] = 0.0
    elem_base_scores["FCCoSp"] = 0.0
    elem_base_scores["CCoSp"] = 0.0
    elem_base_scores["CCoSpV"] = 0.0
    elem_base_scores["FCSSp"] = 0.0
    elem_base_scores["FCoSp"] = 0.0
    elem_base_scores["FSsp2"] = 0.0
    elem_base_scores["LSp"] = 0.0
    elem_base_scores["StSq"] = 0.0
    # Конвертируем ключи словаря в латиницу
    elem_base_scores = {convert_to_latin(
        key): value for key, value in elem_base_scores.items()}
    # Возвращаем словарь
    return elem_base_scores

In [18]:
def clean_elements(row):
    """
    Функция для очистки и нормализации названий элементов фигурного катания.

    Параметры:
    - row (str): строка с названием элемента.

    Возвращает:
    - row (str): очищенное и нормализованное название элемента.
    """
    # Список элементов, для которых не нужно удалять "q"
    tmp_list = ["ChSq1", "ChSq", "ChSq1*", "ChSpl1", "ChSpl", "StSqB",
                "StSq1", "StSq2", "StSq3", "StSq4", "StSq"]
    # Удаление "q", если элемент не в списке tmp_list
    if not any(substring in row for substring in tmp_list):
        row = re.sub("q", "", row)
    # Удаление других символов
    row = re.sub("\*", "", row)
    row = re.sub("<", "", row)
    row = re.sub("<<", "", row)
    row = re.sub("e", "", row)
    row = re.sub("!", "", row)
    # Удаление префиксов "+COMBO", "+SEQ", "+REP"
    if "COMBO" in row:
        row = re.sub("\+COMBO", "", row)
    if "SЕQ" in row:
        row = re.sub("\+SЕQ", "", row)
    if "SEQ" in row:
        row = re.sub("\+SEQ", "", row)
    if "REP" in row:
        row = re.sub("\+REP", "", row)
    # Добавление префикса "1" для одиночных элементов
    if len(row) == 1:
        if row[0] == "T":
            row = re.sub("T", "1T", row)
        elif row[0] == "S":
            row = re.sub("S", "1S", row)
        elif row[0] == "F":
            row = re.sub("F", "1F", row)
        elif row[0] == "A":
            row = re.sub("A", "1A", row)
    elif len(row) == 2:
        if row == "Lz":
            row = re.sub("Lz", "1Lz", row)
        elif row == "Lo":
            row = re.sub("Lo", "1Lo", row)
    # Обработка составных элементов
    if "+" in row:
        row_lst = row.split("+")
        for i in range(len(row_lst)):
            if len(row_lst[i]) == 1:
                if row_lst[i] == "T":
                    row_lst[i] = re.sub("T", "1T", row_lst[i])
                elif row_lst[i] == "S":
                    row_lst[i] = re.sub("S", "1S", row_lst[i])
                elif row_lst[i] == "F":
                    row_lst[i] = re.sub("F", "1F", row_lst[i])
                elif row_lst[i] == "A":
                    row_lst[i] = re.sub("A", "1A", row_lst[i])
            elif len(row_lst[i]) == 2:
                if row_lst[i] == "Lz":
                    row_lst[i] = re.sub("Lz", "1Lz", row_lst[i])
                elif row_lst[i] == "Lo":
                    row_lst[i] = re.sub("Lo", "1Lo", row_lst[i])
        row = "+".join(row_lst)
    # Конвертация в латиницу
    row = convert_to_latin(row)
    # Возврат названия элемента
    return row

In [19]:
def max_base_score_for_clean_elements(row, elem_base_scores):
    """
    Функция для вычисления максимального базового балла для элемента фигурного катания.

    Параметры:
    - row (str): название очищенного элемента.
    - elem_base_scores (dict): словарь с базовыми баллами элементов.

    Возвращает:
    - float: Максимальный базовый балл для данного элемента.
    """
    sum_lst = []

    # Если элемент не составной (не содержит "+")
    if "+" not in row:
        return elem_base_scores[row]
    # Если элемент составной
    else:
        for elem in row.split("+"):
            sum_lst.append(elem_base_scores[elem])
        return sum(sum_lst)

Получаем словарь со скорами.

In [20]:
elem_base_scores = parse_pdf(pdf_dir=ADD_INFO_DIR, file=ELEMS_FILE, convert_to_latin=convert_to_latin)

Вот такой получился словарь.

In [21]:
elem_base_scores

{'1T': 0.4,
 '1S': 0.4,
 '1Lo': 0.5,
 '1Eu': 0.5,
 '1F': 0.5,
 '1Lz': 0.6,
 '1Tq': 0.4,
 '1Sq': 0.4,
 '1Loq': 0.5,
 '1Fq': 0.5,
 '1Lzq': 0.6,
 '1T<': 0.32,
 '1S<': 0.32,
 '1Lo<': 0.4,
 '1F<': 0.4,
 '1Lz<': 0.48,
 '1T<<': 0.0,
 '1S<<': 0.0,
 '1Lo<<': 0.0,
 '1Eu<<': 0.0,
 '1F<<': 0.0,
 '1Lz<<': 0.0,
 '1Fe': 0.4,
 '1Feq': 0.4,
 '1Fe<': 0.3,
 '1Fe<<': 0.0,
 '1Lze': 0.48,
 '1Lzeq': 0.48,
 '1Lze<': 0.36,
 '1Lze<<': 0.0,
 '1F!': 0.5,
 '1F!q': 0.5,
 '1F!<': 0.4,
 '1F!<<': 0.0,
 '1Lz!': 0.6,
 '1Lz!q': 0.6,
 '1Lz!<': 0.48,
 '1Lz!<<': 0.0,
 '1A': 1.1,
 '2T': 1.3,
 '2S': 1.3,
 '2Lo': 1.7,
 '2F': 1.8,
 '2Lz': 2.1,
 '1Aq': 1.1,
 '2Tq': 1.3,
 '2Sq': 1.3,
 '2Loq': 1.7,
 '2Fq': 1.8,
 '2Lzq': 2.1,
 '1A<': 0.88,
 '2T<': 1.04,
 '2S<': 1.04,
 '2Lo<': 1.36,
 '2F<': 1.44,
 '2Lz<': 1.68,
 '1A<<': 0.0,
 '2T<<': 0.4,
 '2S<<': 0.4,
 '2Lo<<': 0.5,
 '2F<<': 0.5,
 '2Lz<<': 0.6,
 '2Fe': 1.44,
 '2Feq': 1.44,
 '2Fe<': 1.08,
 '2Fe<<': 0.4,
 '2Lze': 1.68,
 '2Lzeq': 1.68,
 '2Lze<': 1.26,
 '2Lze<<': 0.48,
 '2F!': 1.8,
 '2

Получаем элементы без ошибок.

In [22]:
data["clean_elems"] = data["title"].apply(clean_elements)

Получаем `max_base_score`.

In [23]:
data["max_base_score"] = data["clean_elems"].apply(
    max_base_score_for_clean_elements, elem_base_scores=elem_base_scores)

In [24]:
data.head(1)

,total_score_id,unit_id,color,tournament_id,date_start,date_end,origin_id,base_score,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,title,decrease,base_score_elements,goe,avg_score,clean_elems,max_base_score
23451,444624,358,green,95,2089-09-20,2089-09-23,1.0,28.13,22.22,49.91,27.69,0.0,5,20,Короткая программа,<< Пониженный прыжок x Надбавка за прыжки во второй половине программы (10%),10,156.61,3Lz+3T<<,<<,7.2,-1.77,5.43,3Lz+3T,10.1


Признаки созданы

### Промежуточный агрегированный датафрейм

Промежуточный агрегированный датафрейм будет сгруппирован по `unit_id` и `tournament_id`. То есть в нем будет информация по юниту и соревнованию.

Столбцы:

🛠 `date_start`: дата начала соревнований,

🛠 `color`: категория,

🛠 `origin_id`: место проведения,

🛠 `base_score_free`: базовый скор для произвольной программы,

🛠 `base_score_short`: базовый скор для короткой программы,

🛠 `components_score_free`: компонентный скор для произвольной программы,

🛠 `components_score_short`: компонентный скор для короткой программы,

🛠 `total_score_free`: тотал скор для произвольной программы,

🛠 `total_score_short`: тотал скор для короткой программы,

🛠 `elements_score_free`: элементный скор для произвольной программы,

🛠 `elements_score_short`: элементный скор для короткой программы,

🛠 `place_free`: место в произвольной программе,

🛠 `place_short`: место в короткой программе,

🛠 `overall_place`: результирующее место,

🛠 `overall_total_score`: результирующий тотал скор,

🛠 `max_base_score_total`: максимальный общий базовый скор,

🛠 `goe_total`: суммарный `goe`,

🛠 `title`: элементы короткой и произвольной программы в одной строке,

🛠 `clean_title`: элементы без короткой и произвольной программы в одной строке.

Словарь для аггрегации.

In [25]:
agg_dict = {
    "date_start": ("date_start", "first"),
    "color": ("color", lambda x: x.mode().iat[0]),
    "origin_id": ("origin_id", "max"),
    "base_score_free": ("base_score", "max"),
    "base_score_short": ("base_score", "min"),
    "components_score_free": ("components_score", "max"),
    "components_score_short": ("components_score", "min"),
    "total_score_free": ("total_score", "max"),
    "total_score_short": ("total_score", "min"),
    "elements_score_free": ("elements_score", "max"),
    "elements_score_short": ("elements_score", "min"),
    "place_free": ("place", "max"),
    "place_short": ("place", "min"),
    "overall_place": ("overall_place", "max"),
    "overall_total_score": ("overall_total_score", "max"),
    "max_base_score_total": ("max_base_score", "sum"),
    "goe_total": ("goe", "sum"),
    "title": ("title", lambda x: "+".join(x)),
    "clean_title": ("clean_elems", lambda x: "+".join(x)),
}

Первичный агрегированный датафрейм

In [26]:
data_agg = data.groupby(["unit_id", "tournament_id"]
                        ).agg(**agg_dict).reset_index()

Пример 5 строк 

In [27]:
data_agg.sample(5)

,unit_id,tournament_id,date_start,color,origin_id,base_score_free,base_score_short,components_score_free,components_score_short,total_score_free,total_score_short,elements_score_free,elements_score_short,place_free,place_short,overall_place,overall_total_score,max_base_score_total,goe_total,title,clean_title
5736,1916,69,2091-04-18,green,1.0,23.37,13.95,28.00,14.90,51.37,29.31,23.87,14.41,3,3,3,80.68,37.1,0.96,StSq1+2Lz+2F+2Lo+CCoSp3+SSp3+1A+2Lz+2F+2S+2Lz+2Lo<+2F+2Lo+2Tq+1A+CCoSp3+ChSq1+FSSp3,StSq1+2Lz+2F+2Lo+CCoSp3+SSp3+1A+2Lz+2F+2S+2Lz+2Lo+2F+2Lo+2T+1A+CCoSp3+ChSq1+FSSp3
4884,1335,72,2091-02-20,green,1.0,58.88,25.69,53.27,25.21,119.95,50.89,66.68,25.68,21,7,11,170.84,83.7,7.79,3Lz+COMBO+2T*+LSp4+3Lz+CCoSp4+3F+2T+CCoSp4+3F+2A+2T+SЕQ+3Lz+3T+3Lo+3S+ChSq1+2A+FCCoSp3+2A+3Lo+FC...,3Lz+2T+LSp4+3Lz+CCoSp4+3F+2T+CCoSp4+3F+2A+2T+3Lz+3T+3Lo+3S+ChSq1+2A+FCCoSp3+2A+3Lo+FCSp2+StSq2+LSp4
3287,745,51,2091-04-25,lime,2.0,27.14,18.64,46.32,21.39,72.38,38.10,27.06,17.71,5,5,5,110.48,55.2,-1.01,CCoSp4+FCSp3+1Lz+2A<+2T+3Lo<<+CSSp4+ChSq1+2A<<+2Lz+1Eu+2S+2F+2T+FCSp2+StSq2+CCoSp3+2Lz+2T+CSSp4+...,CCoSp4+FCSp3+1Lz+2A+2T+3Lo+CSSp4+ChSq1+2A+2Lz+1Eu+2S+2F+2T+FCSp2+StSq2+CCoSp3+2Lz+2T+CSSp4+2A+3Lo
7632,9896,6699,2092-02-23,lime,2.0,36.58,22.63,59.94,29.14,97.23,53.41,38.29,24.27,5,5,5,188.00,56.7,3.10,StSq2+3S+CCoSp4+CSSp4+3T+2T+2A+FSSp4+3S<+3S+2T+CCoSp4+ChSq1+3T+2Lz+3T+2A+2T+SЕQ,StSq2+3S+CCoSp4+CSSp4+3T+2T+2A+FSSp4+3S+3S+2T+CCoSp4+ChSq1+3T+2Lz+3T+2A+2T
351,81,2,2091-03-06,green,1.0,23.90,13.88,32.00,14.72,57.87,30.00,25.87,15.28,5,4,5,87.87,37.2,3.37,FSSp3+ChSq1+CCoSp3+2S+2Lz+2Loq+1A+SЕQ+2Lz+2F+2Lo<+2F+1A+2Lz+CCoSp3+LSp3+StSq1+1A+2F!+2Lo<,FSSp3+ChSq1+CCoSp3+2S+2Lz+2Lo+1A+2Lz+2F+2Lo+2F+1A+2Lz+CCoSp3+LSp3+StSq1+1A+2F+2Lo


### Финальный агрегированный датафрейм

Финальный датафрейм это агрегированная статистика по юниту, а если рассматривать элементы, то все выполненные элементы будут в одной строке за все пройденные соревнования.

Базовые статистики следующие:

🛠 `tournaments`: общее количество соревнований,

🛠 `color`: категория,

🛠 `base_score_free_min`: минимальный базовый скор для произвольной программы,

🛠 `base_score_free_max`: максимальный базовый скор для произвольной программы,

🛠 `base_score_free_avg`: средний базовый скор для произвольной программы,

🛠 `base_score_short_min`: минимальный базовый скор для короткой программы,

🛠 `base_score_short_max`: максимальный базовый скор для короткой программы,

🛠 `base_score_short_avg`: средний базовый скор для короткой программы,

🛠 `components_score_free_min`: минимальный компонентный скор для произвольной программы,

🛠 `components_score_free_max`: максимальный компонентный скор для произвольной программы,

🛠 `components_score_free_avg`: средний компонентный скор для произвольной программы,

🛠 `components_score_short_min`: минимальный компонентный скор для короткой программы,

🛠 `components_score_short_max`: максимальный компонентный скор для короткой программы,

🛠 `components_score_short_avg`: средний компонентный скор для короткой программы,

🛠 `total_score_free_min`: минимальный тотал скор для произвольной программы,

🛠 `total_score_free_max`: максимальный тотал скор для произвольной программы,

🛠 `total_score_free_avg`: средний тотал скор для произвольной программы,

🛠 `total_score_short_min`: минимальный тотал скор для короткой программы,

🛠 `total_score_short_max`: максимальный тотал скор для короткой программы,

🛠 `total_score_short_avg`: средний тотал скор для короткой программы,

🛠 `elements_score_free_min`: минимальный элементный скор для произвольной программы,

🛠 `elements_score_free_max`: максимальный элементный скор для произвольной программы,

🛠 `elements_score_free_avg`: средний элементный скор для произвольной программы,

🛠 `elements_score_short_min`: минимальный элементный скор для короткой программы,

🛠 `elements_score_short_max`: максимальный элементный скор для короткой программы,

🛠 `elements_score_short_avg`: средний элементный скор для короткой программы,

🛠 `place_free_min`: минимальное место в произвольной программе,

🛠 `place_free_max`: максимальное место в произвольной программе,

🛠 `place_short_min`: минимальное место в короткой программе,

🛠 `place_short_max`: максимальное место в короткой программе,

🛠 `overall_place_min`: минимальное итоговое место,

🛠 `overall_place_max`: максимальное итоговое место,

🛠 `overall_total_score_min`: минимальный общий скор,

🛠 `overall_total_score_max`: максимальный общий скор,

🛠 `overall_total_score_avg`: средний общий скор,

🛠 `max_base_score_total_min`: минимальный базовый скор,

🛠 `max_base_score_total_max`: максимальный базовый скор,

🛠 `goe_total_min`: минимальный `goe`,

🛠 `goe_total_max`: максимальный `goe`,

🛠 `title`: исполненные элементы за все соревнования с ошибками,

🛠 `clean_title`: исполненные элементы за все соревнования, которые должны быть чистыми,


Словарь для аггрегации.

In [28]:
final_agg_dict = {
    "tournaments": ("tournament_id", "count"),
    "color": ("color", lambda x: x.mode().iat[0]),
    "base_score_free_min": ("base_score_free", "min"),
    "base_score_free_max": ("base_score_free", "max"),
    "base_score_free_avg": ("base_score_free", "mean"),
    "base_score_short_min": ("base_score_short", "min"),
    "base_score_short_max": ("base_score_short", "max"),
    "base_score_short_avg": ("base_score_short", "mean"),
    "components_score_free_min": ("components_score_free", "min"),
    "components_score_free_max": ("components_score_free", "max"),
    "components_score_free_avg": ("components_score_free", "mean"),
    "components_score_short_min": ("components_score_short", "min"),
    "components_score_short_max": ("components_score_short", "max"),
    "components_score_short_avg": ("components_score_short", "mean"),
    "total_score_free_min": ("total_score_free", "min"),
    "total_score_free_max": ("total_score_free", "max"),
    "total_score_free_avg": ("total_score_free", "mean"),
    "total_score_short_min": ("total_score_short", "min"),
    "total_score_short_max": ("total_score_short", "max"),
    "total_score_short_avg": ("total_score_short", "mean"),
    "elements_score_free_min": ("elements_score_free", "min"),
    "elements_score_free_max": ("elements_score_free", "max"),
    "elements_score_free_avg": ("elements_score_free", "mean"),
    "elements_score_short_min": ("elements_score_short", "min"),
    "elements_score_short_max": ("elements_score_short", "max"),
    "elements_score_short_avg": ("elements_score_short", "mean"),
    "place_free_min": ("place_free", "max"),
    "place_free_max": ("place_free", "min"),
    "place_short_min": ("place_short", "max"),
    "place_short_max": ("place_short", "min"),
    "overall_place_min": ("overall_place", "max"),
    "overall_place_max": ("overall_place", "min"),
    "overall_total_score_min": ("overall_total_score", "min"),
    "overall_total_score_max": ("overall_total_score", "max"),
    "overall_total_score_avg": ("overall_total_score", "mean"),
    "max_base_score_total_min": ("max_base_score_total", "min"),
    "max_base_score_total_max": ("max_base_score_total", "max"),
    "goe_total_min": ("goe_total", "min"),
    "goe_total_max": ("goe_total", "max"),
    "title": ("title", lambda x: "+".join(x)),
    "clean_title": ("clean_title", lambda x: "+".join(x)),
}

Предфинальный датасет.

In [29]:
data_final = data_agg.groupby("unit_id").agg(**final_agg_dict).reset_index()

Пропускаем через конвертер кириллицы в латиницу.

In [30]:
data_final["title"] = data_final["title"].apply(convert_to_latin)
data_final["clean_title"] = data_final["clean_title"].apply(convert_to_latin)

Строка из датасета

In [31]:
data_final.loc[data_final["unit_id"] == 313]

,unit_id,tournaments,color,base_score_free_min,base_score_free_max,base_score_free_avg,base_score_short_min,base_score_short_max,base_score_short_avg,components_score_free_min,components_score_free_max,components_score_free_avg,components_score_short_min,components_score_short_max,components_score_short_avg,total_score_free_min,total_score_free_max,total_score_free_avg,total_score_short_min,total_score_short_max,total_score_short_avg,elements_score_free_min,elements_score_free_max,elements_score_free_avg,elements_score_short_min,elements_score_short_max,elements_score_short_avg,place_free_min,place_free_max,place_short_min,place_short_max,overall_place_min,overall_place_max,overall_total_score_min,overall_total_score_max,overall_total_score_avg,max_base_score_total_min,max_base_score_total_max,goe_total_min,goe_total_max,title,clean_title
212,313,20,lime,49.84,74.02,62.868,23.1,36.95,33.616,58.32,77.42,69.037,28.08,37.41,34.118,109.44,163.73,135.413,52.21,79.95,69.9465,36.76,77.73,61.725,0.0,42.71,30.3515,26,2,19,1,22,2,161.65,239.29,205.3595,72.7,107.6,-3.31,19.25,StSq3+CSSp3+3A+3F!+3Tq+CCoSp4+3Lo+FCSp4+3A+2T+2A+3F!q+3Lo+3Lz+1Eu+3Sq+CCoSp4+3A<<+3F!+REP+CSSp3+...,StSq3+CSSp3+3A+3F+3T+CCoSp4+3Lo+FCSp4+3A+2T+2A+3F+3Lo+3Lz+1Eu+3S+CCoSp4+3A+3F+CSSp3+ChSq1+FCSp4+...


Теперб создаем два варианта столбцов для прыжков:

1. `qty_jumps_...` - для общего количества различных прыжков за все соревнования
2. `mistakes_in__...` - это столбцы для таргета, где `0` - это не было ошибок вообще, `1` - ошибки.

Создаем список с прыжками.

In [32]:
element_cols = [
    "1A",
    "1F",
    "1Lo",
    "1Lz",
    "1S",
    "1T",
    "2A",
    "2F",
    "2Lo",
    "2Lz",
    "2S",
    "2T",
    "3A",
    "3F",
    "3Lo",
    "3Lz",
    "3S",
    "3T",
    "4F",
    "4Lo",
    "4Lz",
    "4S",
    "4T",
    "1Eu",
    "1W",
]

Добывляем столбцы.

In [33]:
for element in element_cols:
    data_final[f"qty_jumps_{element}"] = 0
for element in element_cols:
    data_final[f"mistakes_in_{element}"] = 0

Для подсчета прыжков и разметки таргета написаны функции `total_jumps` и `mistakes_in_jumps` 

In [34]:
def total_jumps(data):
    """
    Функция для подсчета количества прыжковых элементов в данных.

    Параметры:
    - data (pd.DataFrame): данные с информацией о элементах фигурного катания.

    Возвращает:
    - data (pd.DataFrame): исходные данные с добавленными колонками для количества каждого типа прыжков.
    """
    # Шаблон для поиска прыжковых элементов
    pattern = r"(1A|1F|1Lo|1Lz|1S|1T|2A|2F|2Lo|2Lz|2S|2T|3A|3F|3Lo|3Lz|3S|3T|4F|4Lo|4Lz|4S|4T|1Eu|1W)"
    # Создаем список с элементами
    elements = data["clean_title"].split('+')
    # Цикл по списку
    for element in elements:
        # Ищем паттерн
        matches = re.findall(pattern, element)
        # Если паттерн найден
        if matches:
            # Увеличиваем количество типа прыжка на 1
            data[f"qty_jumps_{matches[0]}"] += 1
    # Возвращаем датасет
    return data

In [35]:
def mistakes_in_jumps(data):
    """
    Функция для маркировки таргета.

    Параметры:
    - data (pd.DataFrame): данные с информацией о элементах фигурного катания.

    Возвращает:
    - data (pd.DataFrame): исходные данные с добавленными колонками для количества каждого типа прыжков.
    """
    # Шаблон для поиска прыжковых элементов
    pattern = r"(1A|1F|1Lo|1Lz|1S|1T|2A|2F|2Lo|2Lz|2S|2T|3A|3F|3Lo|3Lz|3S|3T|4F|4Lo|4Lz|4S|4T|1Eu|1W)"
    # Список с вариантами ошибок
    decreasing_lst = ["!", "!*", "!<", "*", "<",
                      "<*", "<<", "<<*", "b", "e", "e*", "q", "q*"]
    # Создаем список с элементами
    elements = data["title"].split('+')
    # Цикл по списку
    for element in elements:
        # Ищем паттерн
        matches = re.findall(pattern, element)
        # Если паттерн найден
        if matches:
            # Если ошибка есть в элементе
            if any(elem in element for elem in decreasing_lst) and "ChSq" not in element:
                # Маркируем таргет
                data[f"mistakes_in_{matches[0]}"] = 1
    # Возвращаем датасет
    return data

Заполняем столбцы

In [36]:
data_final = data_final.apply(total_jumps, axis=1)
data_final = data_final.apply(mistakes_in_jumps, axis=1)

Пример строки финального датафрейма

In [37]:
data_final.loc[(data_final["unit_id"] == 313)]

,unit_id,tournaments,color,base_score_free_min,base_score_free_max,base_score_free_avg,base_score_short_min,base_score_short_max,base_score_short_avg,components_score_free_min,components_score_free_max,components_score_free_avg,components_score_short_min,components_score_short_max,components_score_short_avg,total_score_free_min,total_score_free_max,total_score_free_avg,total_score_short_min,total_score_short_max,total_score_short_avg,elements_score_free_min,elements_score_free_max,elements_score_free_avg,elements_score_short_min,elements_score_short_max,elements_score_short_avg,place_free_min,place_free_max,place_short_min,place_short_max,overall_place_min,overall_place_max,overall_total_score_min,overall_total_score_max,overall_total_score_avg,max_base_score_total_min,max_base_score_total_max,goe_total_min,goe_total_max,title,clean_title,qty_jumps_1A,qty_jumps_1F,qty_jumps_1Lo,qty_jumps_1Lz,qty_jumps_1S,qty_jumps_1T,qty_jumps_2A,qty_jumps_2F,qty_jumps_2Lo,qty_jumps_2Lz,qty_jumps_2S,qty_jumps_2T,qty_jumps_3A,qty_jumps_3F,qty_jumps_3Lo,qty_jumps_3Lz,qty_jumps_3S,qty_jumps_3T,qty_jumps_4F,qty_jumps_4Lo,qty_jumps_4Lz,qty_jumps_4S,qty_jumps_4T,qty_jumps_1Eu,qty_jumps_1W,mistakes_in_1A,mistakes_in_1F,mistakes_in_1Lo,mistakes_in_1Lz,mistakes_in_1S,mistakes_in_1T,mistakes_in_2A,mistakes_in_2F,mistakes_in_2Lo,mistakes_in_2Lz,mistakes_in_2S,mistakes_in_2T,mistakes_in_3A,mistakes_in_3F,mistakes_in_3Lo,mistakes_in_3Lz,mistakes_in_3S,mistakes_in_3T,mistakes_in_4F,mistakes_in_4Lo,mistakes_in_4Lz,mistakes_in_4S,mistakes_in_4T,mistakes_in_1Eu,mistakes_in_1W
212,313,20,lime,49.84,74.02,62.868,23.1,36.95,33.616,58.32,77.42,69.037,28.08,37.41,34.118,109.44,163.73,135.413,52.21,79.95,69.9465,36.76,77.73,61.725,0.0,42.71,30.3515,26,2,19,1,22,2,161.65,239.29,205.3595,72.7,107.6,-3.31,19.25,StSq3+CSSp3+3A+3F!+3Tq+CCoSp4+3Lo+FCSp4+3A+2T+2A+3F!q+3Lo+3Lz+1Eu+3Sq+CCoSp4+3A<<+3F!+REP+CSSp3+...,StSq3+CSSp3+3A+3F+3T+CCoSp4+3Lo+FCSp4+3A+2T+2A+3F+3Lo+3Lz+1Eu+3S+CCoSp4+3A+3F+CSSp3+ChSq1+FCSp4+...,0,2,0,0,0,0,27,0,0,0,4,29,49,35,26,58,16,26,0,0,5,0,0,20,0,0,1,0,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,0,0,1,0,0,0,0


## Подготовка данных для модели

Удаляем ненужные столбцы.

In [38]:
data_final = data_final.drop(["unit_id", "title", "clean_title"], axis=1)

Создаем список колонок с таргетами.

In [39]:
target_cols = [col for col in list(data_final.columns) if "mistakes_in" in col]

Т.к. у нас мультилейбл и возможны различные комбинации для таргета, то простой `train_test_split` тут не подойдет. 

Для разбиения на выборки можно воспользоваться `Multilabel Stratified K-Fold` из библиотеки `iterative-stratification`.

In [40]:
def create_folds(data, target_cols, num_splits):
    """
    Функция для создания фолдов в данных с использованием Multilabel Stratified K-Folds.

    Параметры:
    - data (pd.DataFrame): исходные данные для создания фолдов.
    - target_cols (list): список столбцов с таргетами.
    - num_splits (int): количество фолдов (folds), на которые нужно разделить данные.

    Возвращает:
    - data (pd.DataFrame): Исходные данные с дополнительным столбцом 'kfold', указывающим на номер фолда для каждой строки.
    """
    # инициализация столбца 'kfold' со значением -1 для всех строк данных
    data["kfold"] = -1
    # используем Multilabel Stratified K-Fold для разделения данных на заданное количество фолдов
    mskf = MultilabelStratifiedKFold(
        n_splits=num_splits, shuffle=True, random_state=12345)
    # преобразуем метки в формат numpy array
    data_labels = data[target_cols].values
    # проходим по всем складкам и заполняем столбец 'kfold' соответствующим номером фолда
    for f, (t_, v_) in enumerate(mskf.split(data, data_labels)):
        data.loc[v_, "kfold"] = f
    # возвращаем исходные данные с добавленным столбцом 'kfold'
    return data

Создаем фолды  из расчета 60% на трейн, 20% на валидацию и 20% на тест.

In [41]:
data_final = create_folds(data_final, target_cols, num_splits=5)

Получаем данные для трейн, валид и тест.

In [42]:
train_data = data_final.loc[data_final["kfold"].isin([i for i in range(2, 5)])]
valid_data = data_final.loc[data_final["kfold"] == 0]
test_data = data_final.loc[data_final["kfold"] == 1]

Создаем выборки

In [43]:
X_train = train_data.drop(target_cols+["kfold"], axis=1)
y_train = train_data[target_cols]
X_valid = valid_data.drop(target_cols+["kfold"], axis=1)
y_valid = valid_data[target_cols]
X_test = test_data.drop(target_cols+["kfold"], axis=1)
y_test = test_data[target_cols]

Списки числовых и категориальных переменных для масштабирования и кодирования.

In [44]:
num_cols = X_train.select_dtypes(include=np.number).columns.tolist()
cat_cols = X_train.select_dtypes(exclude=np.number).columns.tolist()

Масштабируем числовые переменные.

In [45]:
scaler = StandardScaler()
scaler.fit(X_train[num_cols])
# масштабирование числовых признаков в трейне, валиде и тесте
X_train[num_cols] = scaler.transform(X_train[num_cols])
X_valid[num_cols] = scaler.transform(X_valid[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

Кодируем категориальные переменные.

In [46]:
encoder = OrdinalEncoder()
encoder.fit(X_train[cat_cols])
# кодирование категориальных признаков в трейне, валиде и тесте
X_train[cat_cols] = encoder.transform(X_train[cat_cols])
X_valid[cat_cols] = encoder.transform(X_valid[cat_cols])
X_test[cat_cols] = encoder.transform(X_test[cat_cols])

Данные готовы.

## Получение предсказаний

Для мультилейбл классификации будет использован `MultiOutputClassifier` совместно с `LGBMClassifier`.

В какчестве метрики взята `Weighted-averaged F1-score`:
- Вычисляется `F1-score` для каждой метки, а затем усредняется с весами, пропорциональными количеству примеров для каждой метки.
- Помогает сбалансировать вклад меток с разным количеством примеров.
- Хорошо подходит, когда важность меток различается.

Инициализируем модель.

In [47]:
model = MultiOutputClassifier(LGBMClassifier(n_estimators=500, max_depth=5, num_leaves=30, learning_rate=0.01, reg_alpha=1, random_state=RANDOM, verbose=-1))

Обучаем.

In [48]:
model.fit(X_train, y_train)

MultiOutputClassifier(estimator=LGBMClassifier(learning_rate=0.01, max_depth=5,
                                               n_estimators=500, num_leaves=30,
                                               random_state=12345, reg_alpha=1,
                                               verbose=-1))

Получаем метрику на валидации.

In [49]:
y_valid_preds = model.predict(X_valid)
f1_score(y_valid, y_valid_preds, average='weighted')

0.7571271142830013

In [50]:
print(classification_report(y_valid, y_valid_preds,
      target_names=target_cols, zero_division=0))

                 precision    recall  f1-score   support

 mistakes_in_1A       0.42      0.14      0.20        37
 mistakes_in_1F       0.60      0.38      0.46         8
mistakes_in_1Lo       0.54      0.41      0.47        17
mistakes_in_1Lz       0.50      0.33      0.40        15
 mistakes_in_1S       0.00      0.00      0.00         2
 mistakes_in_1T       0.00      0.00      0.00         9
 mistakes_in_2A       0.84      0.87      0.86       226
 mistakes_in_2F       0.74      0.69      0.72       149
mistakes_in_2Lo       0.75      0.70      0.73       141
mistakes_in_2Lz       0.74      0.63      0.68       142
 mistakes_in_2S       0.50      0.11      0.19        35
 mistakes_in_2T       0.62      0.53      0.57       148
 mistakes_in_3A       0.80      0.87      0.83        23
 mistakes_in_3F       0.84      0.99      0.91       140
mistakes_in_3Lo       0.82      0.91      0.86       151
mistakes_in_3Lz       0.84      0.98      0.91       147
 mistakes_in_3S       0.85    

Получаем метрику тесте.

In [51]:
y_test_preds = model.predict(X_test)
f1_score(y_test, y_test_preds, average='weighted')

0.7429171232858206

In [52]:
print(classification_report(y_test, y_test_preds,
      target_names=target_cols, zero_division=0))

                 precision    recall  f1-score   support

 mistakes_in_1A       0.29      0.05      0.09        37
 mistakes_in_1F       1.00      0.12      0.22         8
mistakes_in_1Lo       0.31      0.29      0.30        17
mistakes_in_1Lz       0.50      0.40      0.44        15
 mistakes_in_1S       0.00      0.00      0.00         2
 mistakes_in_1T       0.25      0.22      0.24         9
 mistakes_in_2A       0.83      0.83      0.83       229
 mistakes_in_2F       0.72      0.69      0.71       150
mistakes_in_2Lo       0.73      0.72      0.73       141
mistakes_in_2Lz       0.69      0.59      0.64       142
 mistakes_in_2S       0.50      0.11      0.19        35
 mistakes_in_2T       0.61      0.53      0.57       148
 mistakes_in_3A       0.80      0.87      0.83        23
 mistakes_in_3F       0.84      0.98      0.90       140
mistakes_in_3Lo       0.84      0.91      0.87       151
mistakes_in_3Lz       0.87      0.97      0.92       148
 mistakes_in_3S       0.80    

Для наглядности результата можно взять на тесте predict_proba и посмотреть вероятность ошибок.

In [53]:
y_pred_proba = model.predict_proba(X_test)

Пример для первой записи:

Предсказания

In [54]:
y_test_preds[0]

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0])

Реальные значения

In [55]:
y_test.values[0]

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0])

Значение вероятностb для метки 1

In [56]:
y_pred_proba_labels = [y_pred_proba[i][0][1] for i in range(25)]
y_pred_proba_labels

[0.003010476850899771,
 0.0009342659160093754,
 0.0011923055962728584,
 0.0010869983413246752,
 0.01521581786305838,
 0.0011789915244579654,
 0.7281343300593706,
 0.00621620948231089,
 0.565241030433446,
 0.12958539548248915,
 0.0020685036767483854,
 0.6656061087787586,
 0.009052537125546527,
 0.9802391189366245,
 0.8856400164070115,
 0.9389708151706024,
 0.8199921086455599,
 0.9872419291401527,
 0.0012233947370361437,
 0.0006156013262496629,
 0.00099423358617306,
 0.0014844351253208512,
 0.0037220118588095373,
 0.004462835608770479,
 1.0000000036274914e-15]

Результат в виде датафрейма:

In [57]:
# Создаем словарь из списков
data = {
    "real_target": y_test.values[0],
    "predict_taget": y_test_preds[0],
    "proba": y_pred_proba_labels
}

# Создаем датафрейм, используя индекс_список в качестве индексов
df = pd.DataFrame(data, index=target_cols)
df["proba"] = df["proba"].round(2)
df

,real_target,predict_taget,proba
mistakes_in_1A,0,0,0.00
mistakes_in_1F,0,0,0.00
mistakes_in_1Lo,0,0,0.00
mistakes_in_1Lz,0,0,0.00
mistakes_in_1S,0,0,0.02
mistakes_in_1T,0,0,0.00
mistakes_in_2A,1,1,0.73
mistakes_in_2F,0,0,0.01
mistakes_in_2Lo,0,1,0.57
mistakes_in_2Lz,0,0,0.13


Видно что модель чуть-чуть ошиблась и спрогнозировала ошибки в `2Lo` и `2T`, хотя спостсмен их никогда не делал. Стоит на это обратить внимание.

А из классивикационных отчетов видно, что наиболее хорошие метрики там, где есть достачно прыжков и соответсвенно ошибок в них.

## Вывод

В результате исследования была разработана модель способная предсказывать элементы (прыжки), в которых могут быть ошибки. Соотвественно стоит обращать внимание на предсказание модели, и даже если она предскажет ошибку в стабильно исполняемом прыжке, то стоит обраттить внимание на доп. отработку элемента.

В качестве модели использовался  `MultiOutputClassifier` на основе `LGBMClassifier`.

Метрика `Weighted-averaged F1-score` на тестовой выборке - 0.7459, что является достаточно высоким показателем.

Как вариант улучшения модели можно рассмотреть принципиально более расширенное формирование исходных данных, например добавить рейтинг спортсмена, более подробная расшифровка снятия баллов в элементах и т.п.